In [2]:
# import sys
# sys.path.append('../bicm/')
# from BipartiteGraph import *
# import bicm
from bicm import BipartiteGraph
import numpy as np

This test uses a plant-pollinator network from Petanidou, T. (1991). Pollination ecology in a phryganic ecosystem.
This script is run on an ordinary laptop.

In [3]:
biad_mat_names = np.loadtxt('test_dataset.csv', delimiter=',', dtype=str)
plants = biad_mat_names[1:,0]
pollinators = biad_mat_names[0, 1:]
biad_mat = biad_mat_names[1:, 1:].astype(np.ubyte)
plants_dict = dict(enumerate(np.unique(plants)))
plants_inv_dict = {v:k for k,v in plants_dict.items()}
pollinators_dict = dict(enumerate(np.unique(pollinators)))
pollinators_inv_dict = {v:k for k,v in pollinators_dict.items()}

You can manipulate a data structure into another with the functions of the package. It supports biadjacency matrices (numpy arrays or scipy sparse matrices), edgelists, adjacency lists (dictionaries) or just the degree sequences.

Let's create an edgelist with names.

In [4]:
# These data type converters return additional objects, keeping track of the node labeling and the node degrees.
# The desired data structure is always at index 0. Check the documentation for more info.
edgelist = edgelist_from_biadjacency(biad_mat)[0]
edgelist_names = [(plants_dict[edge[0]], pollinators_dict[edge[1]]) for edge in edgelist]
print(edgelist_names[:5])

[('"Acanthus spinosus"', '"Acmaeodera bipunctata "'), ('"Acanthus spinosus"', '"Adia cinerella "'), ('"Acanthus spinosus"', '"Alophora pusilla"'), ('"Acanthus spinosus"', '"Amasis lateralis"'), ('"Acanthus spinosus"', '"Amasis similis"')]


Now we can use the edgelist to create the bipartite graph. In two ways:

In [5]:
myGraph = BipartiteGraph(edgelist=edgelist_names)

In [6]:
myGraph = BipartiteGraph()
myGraph.set_edgelist(edgelist_names)

And now let's simply compute the bicm! This should run instantly. The solver checks that the solution is correct automatically.

In [7]:
myGraph.solve_bicm()
dict_x, dict_y = myGraph.get_bicm_fitnesses()
print('Yielded data type is:', type(dict_x))

max rows error = 5.684341886080802e-14
max columns error = 2.842170943040401e-14
total error = 2.029265644409861e-12
Solver converged.
Yielded data type is: <class 'dict'>


The computed fitnesses are given as dictionaries, keeping track of the name of the nodes. If you are sure about the order of the fitnesses, you can get separately the fitness vectors and the dictionaries that keep track of the order of the nodes:

In [8]:
x = myGraph.x
y = myGraph.y
rows_dict = myGraph.rows_dict
cols_dict = myGraph.cols_dict

Alternatively, you can have the bicm matrix of probabilities, whose entries i, j are the bicm probabilities of having a link between nodes i and j (who will correspond to rows_dict[i] and cols_dict[j]), or compute it on your own

In [9]:
avg_mat = myGraph.get_bicm_matrix()
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


In [10]:
avg_mat = bicm_from_fitnesses(x, y)
print(avg_mat[0, 0] == x[0] * y[0] / (1 + x[0] * y[0]))

True


And, if you like, you can check the solution by yourself:

In [11]:
# The data type conversions are generally consistent with the order, so this works
check_sol(biad_mat, avg_mat)

max rows error = 5.684341886080802e-14
max columns error = 6.394884621840902e-14
total error = 3.3653080322437745e-12


You can then sample from the BiCM ensemble in two ways, via matrix sampling or edgelist sampling.

In [12]:
# Here we generate a sampling of 1000 networks and calculate the distribution of the number of links.
tot_links = []
for sample_i in range(1000):
    sampled_network = sample_bicm(avg_mat)
    tot_links.append(np.sum(sampled_network))

In [13]:
sampled_edgelist = sample_bicm_edgelist(x, y)

Now let's compute the projection: it can take some seconds.

In [14]:
rows_projection = myGraph.get_rows_projection()
print(rows_projection)


[['"Asphodelus aestivus"' '"Astragalus monspessulanus"']
 ['"Asphodelus aestivus"' '"Biscutella didyma"']
 ['"Astragalus monspessulanus"' '"Biscutella didyma"']
 ['"Carduus pycnocephalus"' '"Hirschfeldia incana"']
 ['"Centaurea raphanina"' '"Echinops microcephalus"']
 ['"Centaurea raphanina"' '"Fumana thymifolia"']
 ['"Echinops microcephalus"' '"Fumana thymifolia"']
 ['"Fritillaria graeca"' '"Heliotropium dolosum"']
 ['"Fritillaria graeca"' '"Hypochoeris achyrophorus"']
 ['"Fumana arabica"' '"Papaver rhoeas"']
 ['"Heliotropium dolosum"' '"Hypochoeris achyrophorus"']
 ['"Phlomis fructicosa"' '"Plantago lagopus"']
 ['"Phlomis fructicosa"' '"Prasium majus"']
 ['"Plantago lagopus"' '"Prasium majus"']]


If you don't want to get the progress_bar you can set progress_bar=False. If you want to re-compute the projection with different settings, use compute_projection()

In [15]:
myGraph.compute_projection(rows=True, alpha=0.01, progress_bar=False)

In [16]:
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

[['"Asphodelus aestivus"' '"Astragalus monspessulanus"']
 ['"Asphodelus aestivus"' '"Biscutella didyma"']
 ['"Astragalus monspessulanus"' '"Biscutella didyma"']
 ['"Centaurea raphanina"' '"Echinops microcephalus"']
 ['"Centaurea raphanina"' '"Fumana thymifolia"']
 ['"Echinops microcephalus"' '"Fumana thymifolia"']
 ['"Fumana arabica"' '"Papaver rhoeas"']
 ['"Phlomis fructicosa"' '"Plantago lagopus"']
 ['"Phlomis fructicosa"' '"Prasium majus"']
 ['"Plantago lagopus"' '"Prasium majus"']]


These projection only contain links between nodes that behave similarly with respect to the expected behavior calculated from the BiCM. They could also be empty:

In [17]:
cols_projection = myGraph.get_cols_projection()


No V-motifs will be validated. Try increasing alpha


In [18]:
# Increasing alpha too much will make the projection too dense. 
# It is not suggested: if it is empty with a high alpha, try different projections.
# Check the documentation and the papers for more info.
myGraph.compute_projection(rows=False, alpha=0.2)


No V-motifs will be validated. Try increasing alpha


Different projection methods are implemented. 
They differ in the way of approximating the poisson-binomial distribution of the similarities between nodes.
By default, the poisson approximation is used, since the poisson binomial is very expensive to compute. Check the docs for more!
Here, an example of the poibin implementation. This can take even some minutes, and it's faster when using a biadjacency matrix instead of other data types. Note that the poisson approximation is fairly good.

In [19]:
myGraph.compute_projection(rows=False, method='poibin')
rows_projection = myGraph.get_rows_projection()
print(rows_projection)

Building the V-motifs list...
Calculating p-values...



No V-motifs will be validated. Try increasing alpha
[['"Asphodelus aestivus"' '"Astragalus monspessulanus"']
 ['"Asphodelus aestivus"' '"Biscutella didyma"']
 ['"Astragalus monspessulanus"' '"Biscutella didyma"']
 ['"Centaurea raphanina"' '"Echinops microcephalus"']
 ['"Centaurea raphanina"' '"Fumana thymifolia"']
 ['"Echinops microcephalus"' '"Fumana thymifolia"']
 ['"Fumana arabica"' '"Papaver rhoeas"']
 ['"Phlomis fructicosa"' '"Plantago lagopus"']
 ['"Phlomis fructicosa"' '"Prasium majus"']
 ['"Plantago lagopus"' '"Prasium majus"']]


Now, let's have fun with other data structures and solver methods.

In [20]:
adj_list_names = adjacency_list_from_edgelist(edgelist_names, convert_type=False)[0]

In [21]:
myGraph2 = BipartiteGraph(adjacency_list=adj_list_names)

Asking for the projection immediately will do everything automatically.

In [22]:
myGraph2.get_rows_projection()

First I have to compute the BiCM. Computing...
max rows error = 5.684341886080802e-14
max columns error = 2.842170943040401e-14
total error = 2.029265644409861e-12
Solver converged.


array([['"Asphodelus aestivus"', '"Astragalus monspessulanus"'],
       ['"Asphodelus aestivus"', '"Biscutella didyma"'],
       ['"Astragalus monspessulanus"', '"Biscutella didyma"'],
       ['"Carduus pycnocephalus"', '"Hirschfeldia incana"'],
       ['"Centaurea raphanina"', '"Echinops microcephalus"'],
       ['"Centaurea raphanina"', '"Fumana thymifolia"'],
       ['"Echinops microcephalus"', '"Fumana thymifolia"'],
       ['"Fritillaria graeca"', '"Heliotropium dolosum"'],
       ['"Fritillaria graeca"', '"Hypochoeris achyrophorus"'],
       ['"Fumana arabica"', '"Papaver rhoeas"'],
       ['"Heliotropium dolosum"', '"Hypochoeris achyrophorus"'],
       ['"Phlomis fructicosa"', '"Plantago lagopus"'],
       ['"Phlomis fructicosa"', '"Prasium majus"'],
       ['"Plantago lagopus"', '"Prasium majus"']], dtype='<U27')

In [23]:
rows_deg = biad_mat.sum(1)
cols_deg = biad_mat.sum(0)

In [24]:
myGraph3 = BipartiteGraph(degree_sequences=(rows_deg, cols_deg))

Default (recommended) method is 'newton', three more methods are implemented

In [25]:
myGraph3.solve_bicm(method='fixed-point', tolerance=1e-5, exp=True)

max rows error = 1.5070042991283117e-09
max columns error = 4.630766170521383e-08
total error = 9.787269128480602e-08
Solver converged.


In [26]:
myGraph3.solve_bicm(method='quasinewton', tolerance=1e-12, exp=True)

max rows error = 5.133949088076406e-07
max columns error = 4.904185857412813e-07
total error = 1.090741872067369e-05
Solver converged.


This is the only method that works with a scipy solver, not recommended.

In [27]:
myGraph3.solve_bicm(method='root')

max rows error = 2.842170943040401e-14
max columns error = 1.4210854715202004e-14
total error = 9.79216707719388e-13
Solver converged.


If you want to retrieve the p-values of the projection on one layer, use one of the following functions:

In [ ]:
# Use the rows arg to project on the rows or columns
adj_list = adjacency_list_from_adjacency_list(adj_list_names)[0]
pval_list = projection_calculator_light(edgelist, x, y, return_pvals=True, rows=True)
pval_list = projection_calculator_light_from_adjacency(adj_list, x, y, return_pvals=True)
pval_list = projection_calculator(biad_mat, avg_mat, return_pvals=True)

Finally, if you want to manage the calculation of the pvalues for the projection on your own, you can use the class PvaluesHandler (see more in the docs)

In [28]:
# For the projection on the rows layer:
v_list = vmotifs_from_adjacency_list(adj_list)
pval_obj = PvaluesHandler()
pval_obj.set_fitnesses(x, y)
pval_obj.compute_pvals(v_list, method='poisson', threads_num=4, progress_bar=True)
pval_list = pval_obj.pval_list
# Contains a list of the couples of nodes that share at least a neighbor with the respective pvalue
print(pval_list[:5])


[(0, 1, 0.9972317357273861), (0, 2, 0.9999999782267512), (0, 3, 0.9923305795538505), (0, 4, 0.9691310327759), (0, 5, 0.9958459965121919)]


In [29]:
# For the projection on the opposite layer:
inv_adj_list = adjacency_list_from_adjacency_list(adj_list_names)[1]
v_list = vmotifs_from_adjacency_list(inv_adj_list)
pval_obj = PvaluesHandler()
pval_obj.set_fitnesses(y, x)
pval_obj.compute_pvals(v_list, method='poisson', threads_num=4, progress_bar=True)
pval_list = pval_obj.pval_list
# Contains a list of the couples of nodes that share at least a neighbor with the respective pvalue
print(pval_list[:5])


[(0, 1, 0.4713915751929158), (0, 2, 0.33940164225865216), (0, 3, 0.44956043489541525), (0, 4, 0.3781962354748192), (0, 5, 0.6687479057447224)]
